In [16]:
import numpy as np
import plotly.graph_objects as go

In [17]:
# initializing the probability density vector

def initialize_probability_vector(initial_true_value, std_dev):
    """
    Initialize the probability vector for the market maker model.

    Args:
    initial_true_value (float): The initial estimate of the true value of the security.
    std_dev (float): The standard deviation around the initial true value.

    Returns:
    numpy.ndarray: A probability vector of size 101, representing probabilities for each price value from 0 to 100.
    """
    # Define the range of possible price values (0 to 100)
    price_values = np.arange(101)

    # Calculate the probability of each price value based on a normal distribution centered around the initial true value
    probabilities = np.exp(-0.5 * ((price_values - initial_true_value) / std_dev) ** 2)
    probabilities /= probabilities.sum()  # Normalize to ensure the sum of probabilities is 1

    return probabilities

# Example usage
initial_true_value = 50  # Example initial true value
std_dev = 10  # Example standard deviation
probability_vector = initialize_probability_vector(initial_true_value, std_dev)

# plot the probability vector
fig = go.Figure(data=[go.Scatter(x=np.arange(101), y=probability_vector)])
fig.update_layout(title="Probability Vector", xaxis_title="Price", yaxis_title="Probability")
fig.show()

In [18]:
# updating the probability density vector given a buy order

import math

def normal_cdf(x, mean, std_dev):
    """
    Compute the cumulative distribution function (CDF) for a normal distribution.

    Args:
    x (float): The point at which to evaluate the CDF.
    mean (float): The mean of the normal distribution.
    std_dev (float): The standard deviation of the normal distribution.

    Returns:
    float: The value of the CDF at x.
    """
    return 0.5 * (1 + math.erf((x - mean) / (std_dev * math.sqrt(2))))

def probability_buy_given_V(Vi, Pa, alpha, eta, sigma_W):
    """
    Compute the probability of a buy order given the true value V = Vi.

    Args:
    Vi (float): The potential true value of the security.
    Pa (float): The current ask price set by the market maker.
    alpha (float): The proportion of informed traders in the market.
    eta (float): The base probability of a trade happening.
    sigma_W (float): The standard deviation of the informed traders' signal noise.

    Returns:
    float: The probability of observing a buy order given V = Vi.
    """
    if Vi <= Pa:
        # Calculate the probability for the case where Vi <= Pa
        return (1 - alpha) * eta + alpha * (1 - normal_cdf(Pa - Vi, 0, sigma_W))
    else:
        # Calculate the probability for the case where Vi > Pa
        return (1 - alpha) * eta + alpha * normal_cdf(Vi - Pa, 0, sigma_W)

# Example usage
Vi = 45  # Example value of Vi
Pa = 50  # Example ask price
alpha = 0.3  # Example proportion of informed traders
eta = 0.1  # Example base probability of a trade happening
sigma_W = 5  # Example standard deviation of informed traders' signal noise

probability_buy = probability_buy_given_V(Vi, Pa, alpha, eta, sigma_W)
probability_buy

0.11759657617943711

In [19]:
# updating the probability density vector given a sell order

def probability_sell_given_V(Vi, Pb, alpha, eta, sigma_W):
    """
    Compute the probability of a sell order given the true value V = Vi.

    Args:
    Vi (float): The potential true value of the security.
    Pb (float): The current bid price set by the market maker.
    alpha (float): The proportion of informed traders in the market.
    eta (float): The base probability of a trade happening.
    sigma_W (float): The standard deviation of the informed traders' signal noise.

    Returns:
    float: The probability of observing a sell order given V = Vi.
    """
    if Vi >= Pb:
        # Calculate the probability for the case where Vi >= Pb
        return (1 - alpha) * eta + alpha * (1 - normal_cdf(Vi - Pb, 0, sigma_W))
    else:
        # Calculate the probability for the case where Vi < Pb
        return (1 - alpha) * eta + alpha * normal_cdf(Pb - Vi, 0, sigma_W)

# Example usage
Vi = 55  # Example value of Vi
Pb = 50  # Example bid price
alpha = 0.3  # Example proportion of informed traders
eta = 0.1  # Example base probability of a trade happening
sigma_W = 5  # Example standard deviation of informed traders' signal noise

probability_sell = probability_sell_given_V(Vi, Pb, alpha, eta, sigma_W)
probability_sell


0.11759657617943711

In [20]:
def probability_V_given_buy(Vi, probability_vector, probability_buy, total_buy_probability):
    """
    Compute the posterior probability P(V=Vi | Buy).

    Args:
    Vi (float): The potential true value of the security.
    probability_vector (numpy.ndarray): The current probability vector for all possible values of V.
    probability_buy (float): The probability of a buy order given V = Vi.
    total_buy_probability (float): The total probability of observing a buy order, summed over all V.

    Returns:
    float: The posterior probability P(V=Vi | Buy).
    """
    prior_probability = probability_vector[Vi]
    return (probability_buy * prior_probability) / total_buy_probability

# Example usage
probability_vector = initialize_probability_vector(50, 10)  # Re-initialize the probability vector for this example
Vi = 45  # Example value of Vi
probability_buy_Vi = probability_buy_given_V(Vi, 50, 0.3, 0.1, 5)  # Calculate Pr(Buy | V=Vi) for this example
total_buy_probability = sum(probability_buy_given_V(i, 50, 0.3, 0.1, 5) for i in range(101))  # Sum over all possible V

posterior_probability = probability_V_given_buy(Vi, probability_vector, probability_buy_Vi, total_buy_probability)
posterior_probability

0.0001863262606232847

In [21]:
def probability_V_given_sell(Vi, probability_vector, probability_sell, total_sell_probability):
    """
    Compute the posterior probability P(V=Vi | Sell).

    Args:
    Vi (float): The potential true value of the security.
    probability_vector (numpy.ndarray): The current probability vector for all possible values of V.
    probability_sell (float): The probability of a sell order given V = Vi.
    total_sell_probability (float): The total probability of observing a sell order, summed over all V.

    Returns:
    float: The posterior probability P(V=Vi | Sell).
    """
    prior_probability = probability_vector[Vi]
    return (probability_sell * prior_probability) / total_sell_probability

# Example usage
probability_vector = initialize_probability_vector(50, 10)  # Re-initialize the probability vector for this example
Vi = 55  # Example value of Vi
probability_sell_Vi = probability_sell_given_V(Vi, 50, 0.3, 0.1, 5)  # Calculate Pr(Sell | V=Vi) for this example
total_sell_probability = sum(probability_sell_given_V(i, 50, 0.3, 0.1, 5) for i in range(101))  # Sum over all possible V

posterior_probability = probability_V_given_sell(Vi, probability_vector, probability_sell_Vi, total_sell_probability)
posterior_probability

0.00018632626062328479

# Getting Pb and Pa

In [34]:
# Function to calculate the expected value E[V]
def calculate_expected_value(pdf, possible_prices):
    return np.sum(pdf * possible_prices)

# Function to 'cycle down' from E[V] to find the bid price
def calculate_bid(pdf, expected_value, possible_prices):
    min_diff = float('inf')
    bid_price = 0

    for price in reversed(possible_prices):
        if price > expected_value:
            continue

        # Calculate the left and right hand sides of the equation
        left_hand_side = np.sum(pdf[possible_prices <= price] * possible_prices[possible_prices <= price])
        right_hand_side = np.sum(pdf[possible_prices > price] * possible_prices[possible_prices > price])

        # Calculate the difference between the two sides
        diff = abs(right_hand_side - left_hand_side)

        # Update the bid price if this is the smallest difference so far
        if diff < min_diff:
            min_diff = diff
            bid_price = price
        else:
            # Stop cycling down if the difference starts increasing
            break

    return bid_price

# Calculate the expected value E[V]
pdf = initialize_probability_vector(50, 10)
possible_prices = np.arange(101)
expected_value = calculate_expected_value(pdf, possible_prices)

# Find the bid price using the 'cycling down' approach
bid_price_cycling_down = calculate_bid(pdf, expected_value, possible_prices)
bid_price_cycling_down

49

In [35]:
# Function to 'cycle up' from E[V] to find the ask price
def calculate_ask(pdf, expected_value, possible_prices):
    min_diff = float('inf')
    ask_price = 0

    for price in possible_prices:
        if price < expected_value:
            continue

        # Calculate the left and right hand sides of the equation
        left_hand_side = np.sum(pdf[possible_prices <= price] * possible_prices[possible_prices <= price])
        right_hand_side = np.sum(pdf[possible_prices > price] * possible_prices[possible_prices > price])

        # Calculate the difference between the two sides
        diff = abs(right_hand_side - left_hand_side)

        # Update the ask price if this is the smallest difference so far
        if diff < min_diff:
            min_diff = diff
            ask_price = price
        else:
            # Stop cycling up if the difference starts increasing
            break

    return ask_price

# Calculate the expected value E[V]
pdf = initialize_probability_vector(50, 10)
possible_prices = np.arange(101)
expected_value = calculate_expected_value(pdf, possible_prices)

# Find the ask price using the 'cycling up' approach
ask_price_cycling_up = calculate_ask(pdf, expected_value, possible_prices)
ask_price_cycling_up

51

# Testing

In [70]:
# INITIALIZE VECTOR

initial_probability_vector = initialize_probability_vector(initial_true_value=50, std_dev=10)

# Plot the initial probability vector

fig = go.Figure(data=[go.Scatter(x=np.arange(101), y=initial_probability_vector)])
fig.update_layout(title="Initial Probability Vector", xaxis_title="Price", yaxis_title="Probability")
fig.show()

In [58]:
# COMPUTE BID AND ASK PRICES

PRICES = np.arange(101)

Pb = calculate_bid(initial_probability_vector, calculate_expected_value(initial_probability_vector, PRICES), PRICES)
Pa = calculate_ask(initial_probability_vector, calculate_expected_value(initial_probability_vector, PRICES), PRICES)

print("Bid price: ", Pb)
print("Ask price: ", Pa)

Bid price:  49
Ask price:  51


In [59]:
# BUY ORDER SIMULATION
ALPHA = 0.3
ETA = 0.1
SIGMA_W = 5

# Calculate total probability of a buy order
total_buy_probability = sum(probability_buy_given_V(Vi, Pa, ALPHA, ETA, SIGMA_W) * initial_probability_vector[Vi] 
                            for Vi in range(101))

# Update the probability vector given a buy order
updated_probability_vector = np.zeros(101)
for i in range(101):
    p_buy = probability_buy_given_V(i, Pa, ALPHA, ETA, SIGMA_W)
    updated_probability_vector[i] = probability_V_given_buy(i, initial_probability_vector, p_buy, total_buy_probability)

# Recalculate the expected true value
expected_true_value = calculate_expected_value(updated_probability_vector, np.arange(101))

# Assuming calculate_bid and calculate_ask functions are defined and correct
Pb = calculate_bid(updated_probability_vector, expected_true_value, np.arange(101))
Pa = calculate_ask(updated_probability_vector, expected_true_value, np.arange(101))

print("Expected True Value: ", expected_true_value)
print("Bid price: ", Pb)
print("Ask price: ", Pa)


Expected True Value:  55.09388309130345
Bid price:  55
Ask price:  56


In [ ]:
# Assuming the initial probability vector and necessary functions are defined correctly

# Parameters
V_min = 0
V_max = 100
initial_true_value = 50
std_dev = 10
Pa = 55  # Example initial ask price

# Initialize the probability vector
probability_vector = initialize_probability_vector(initial_true_value, std_dev, V_min, V_max)

# Process 100 buy orders through the market maker
for _ in range(100):
    total_buy_probability = sum(probability_buy_given_V(Vi, Pa, ALPHA, ETA, SIGMA_W) * probability_vector[Vi] 
                                for Vi in range(V_min, V_max + 1))

    updated_probability_vector = np.zeros(V_max - V_min + 1)
    for i in range(V_min, V_max + 1):
        p_buy = probability_buy_given_V(i, Pa, ALPHA, ETA, SIGMA_W)
        updated_probability_vector[i] = probability_V_given_buy(i, probability_vector, p_buy, total_buy_probability)
    
    probability_vector = updated_probability_vector  # Update the main probability vector

# Plot the updated probability vector
import plotly.graph_objects as go

fig = go.Figure(data=[go.Scatter(x=np.arange(V_min, V_max + 1), y=probability_vector)])
fig.update_layout(title="Probability Vector after 100 Buy Orders", xaxis_title="Price", yaxis_title="Probability")
fig.show()



In [62]:
# MANY BUY ORDERS SIMULATION

# update the probability vector given a buy order
last_probability_vector = initial_probability_vector

for i in range(100):

    updated_probability_vector = np.zeros(101)
    Pa = calculate_ask(last_probability_vector, calculate_expected_value(last_probability_vector, PRICES), PRICES)
    total_buy_probability = sum(probability_buy_given_V(Vi, Pa, ALPHA, ETA, SIGMA_W) * last_probability_vector[Vi]
                                    for Vi in range(101))

    for i in range(len(last_probability_vector)):
        previous_probability = last_probability_vector[i]
        Vi = i
        p_buy = probability_buy_given_V(Vi, Pa, ALPHA, ETA, SIGMA_W)

        new_probability = probability_V_given_buy(Vi, last_probability_vector, p_buy, total_buy_probability)
        updated_probability_vector[i] = new_probability

    last_probability_vector = updated_probability_vector

# Plot the updated probability vector
fig = go.Figure(data=[go.Scatter(x=np.arange(101), y=last_probability_vector)])
fig.update_layout(title="Updated Probability Vector", xaxis_title="Price", yaxis_title="Probability")
fig.show()

# UPDATE BID AND ASK

Pb = calculate_bid(last_probability_vector, calculate_expected_value(last_probability_vector, PRICES), PRICES)
Pa = calculate_ask(last_probability_vector, calculate_expected_value(last_probability_vector, PRICES), PRICES)

print("EV: ", calculate_expected_value(last_probability_vector, PRICES))
print("Bid price: ", Pb)
print("Ask price: ", Pa)

EV:  99.99973546008948
Bid price:  99
Ask price:  100


In [65]:
# SELL ORDER SIMULATION

# update the probability vector given a sell order
updated_probability_vector = np.zeros(101)

total_sell_probability = sum(probability_sell_given_V(Vi, Pb, ALPHA, ETA, SIGMA_W) * initial_probability_vector[Vi]
                                for Vi in range(101))

for i in range(len(initial_probability_vector)):
    previous_probability = initial_probability_vector[i]
    Vi = i
    p_sell = probability_sell_given_V(Vi, Pb, ALPHA, ETA, SIGMA_W)
    new_probability = probability_V_given_sell(Vi, initial_probability_vector, p_sell, total_sell_probability)
    updated_probability_vector[i] = new_probability

# Plot the updated probability vector
fig = go.Figure(data=[go.Scatter(x=np.arange(101), y=updated_probability_vector)])
fig.update_layout(title="Updated Probability Vector", xaxis_title="Price", yaxis_title="Probability")
fig.show()

# UPDATE BID AND ASK

Pb = calculate_bid(updated_probability_vector, calculate_expected_value(updated_probability_vector, PRICES), PRICES)
Pa = calculate_ask(updated_probability_vector, calculate_expected_value(updated_probability_vector, PRICES), PRICES)

print("EV: ", calculate_expected_value(updated_probability_vector, PRICES))
print("Bid price: ", Pb)
print("Ask price: ", Pa)

EV:  45.3782310663875
Bid price:  45
Ask price:  46


In [66]:
# MANY SELL ORDERS SIMULATION

# update the probability vector given a sell order
last_probability_vector = initial_probability_vector

for i in range(100):

    updated_probability_vector = np.zeros(101)
    Pb = calculate_bid(last_probability_vector, calculate_expected_value(last_probability_vector, PRICES), PRICES)
    total_sell_probability = sum(probability_sell_given_V(Vi, Pb, ALPHA, ETA, SIGMA_W) * last_probability_vector[Vi]
                                    for Vi in range(101))

    for i in range(len(last_probability_vector)):
        previous_probability = last_probability_vector[i]
        Vi = i
        p_sell = probability_sell_given_V(Vi, Pb, ALPHA, ETA, SIGMA_W)

        new_probability = probability_V_given_sell(Vi, last_probability_vector, p_sell, total_sell_probability)
        updated_probability_vector[i] = new_probability

    last_probability_vector = updated_probability_vector

# Plot the updated probability vector
fig = go.Figure(data=[go.Scatter(x=np.arange(101), y=last_probability_vector)])
fig.update_layout(title="Updated Probability Vector", xaxis_title="Price", yaxis_title="Probability")
fig.show()

# UPDATE BID AND ASK

Pb = calculate_bid(last_probability_vector, calculate_expected_value(last_probability_vector, PRICES), PRICES)
Pa = calculate_ask(last_probability_vector, calculate_expected_value(last_probability_vector, PRICES), PRICES)

print("EV: ", calculate_expected_value(last_probability_vector, PRICES))
print("Bid price: ", Pb)
print("Ask price: ", Pa)

EV:  0.00026453991051869066
Bid price:  0
Ask price:  1


# More realistic simulation

In [74]:
# See what happens when 75% of the orders are buy and 25% are sell

# update the probability vector given a sell order
last_probability_vector = initial_probability_vector

for i in range(100):

    updated_probability_vector = np.zeros(101)

    # generate a random number between 1 and 4
    random_number = np.random.randint(1, 5)

    if random_number in [1, 2, 3]:
        # 75% chance of a buy order
        Pa = calculate_ask(last_probability_vector, calculate_expected_value(last_probability_vector, PRICES), PRICES)
        total_buy_probability = sum(probability_buy_given_V(Vi, Pa, ALPHA, ETA, SIGMA_W) * last_probability_vector[Vi]
                                    for Vi in range(101))

        for i in range(len(last_probability_vector)):
            previous_probability = last_probability_vector[i]
            Vi = i
            p_buy = probability_buy_given_V(Vi, Pa, ALPHA, ETA, SIGMA_W)

            new_probability = probability_V_given_buy(Vi, last_probability_vector, p_buy, total_buy_probability)
            updated_probability_vector[i] = new_probability

    else:
        # 25% chance of a sell order
        Pb = calculate_bid(last_probability_vector, calculate_expected_value(last_probability_vector, PRICES), PRICES)
        total_sell_probability = sum(probability_sell_given_V(Vi, Pb, ALPHA, ETA, SIGMA_W) * last_probability_vector[Vi]
                                    for Vi in range(101))

        for i in range(len(last_probability_vector)):
            previous_probability = last_probability_vector[i]
            Vi = i
            p_sell = probability_sell_given_V(Vi, Pb, ALPHA, ETA, SIGMA_W)

            new_probability = probability_V_given_sell(Vi, last_probability_vector, p_sell, total_sell_probability)
            updated_probability_vector[i] = new_probability

    last_probability_vector = updated_probability_vector

# Plot the updated probability vector
fig = go.Figure(data=[go.Scatter(x=np.arange(101), y=last_probability_vector)])
fig.update_layout(title="Updated Probability Vector", xaxis_title="Price", yaxis_title="Probability")
fig.show()

# UPDATE BID AND ASK

Pb = calculate_bid(last_probability_vector, calculate_expected_value(last_probability_vector, PRICES), PRICES)
Pa = calculate_ask(last_probability_vector, calculate_expected_value(last_probability_vector, PRICES), PRICES)

print("EV: ", calculate_expected_value(last_probability_vector, PRICES))
print("Bid price: ", Pb)
print("Ask price: ", Pa)

EV:  85.04904233302375
Bid price:  84
Ask price:  86
